<a href="https://colab.research.google.com/github/yapyang040527/Yang1141/blob/main/HW1_%E6%97%A5%E5%B8%B8%E6%94%AF%E5%87%BA%E9%80%9F%E7%AE%97%E8%88%87%E5%88%86%E6%94%A4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 日常支出速算與分攤（作業一）


*   目標：從 Sheet 讀「消費紀錄」→ 計總額/分類小計/AA 分攤 → 寫回 Sheet Summary 分頁。

*  AI 點子（可選）：請模型總結本週花錢習慣與建議（例如「外食過多」）。
*  Sheet 欄位: date, category, item, amount, payer



GoogleSheet: ttps://docs.google.com/spreadsheets/d/1gtKuA4gHxt3JQgH9SV9St_7KtBb3YjdjVMsx7nLhYEw/edit?gid=0#gid=0

In [5]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [6]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

import pandas as pd
# read data and put it in a dataframe
# 在 google 工作表載入 gsheets
gsheets = gc.open_by_url('https://docs.google.com/spreadsheets/d/1gtKuA4gHxt3JQgH9SV9St_7KtBb3YjdjVMsx7nLhYEw/edit?gid=0#gid=0')

In [7]:
# 從 gsheets 的 All-whiteboard-device 載入 sheets
sheets = gsheets.worksheet('工作表1').get_all_values()

# 將 sheets1 資料載入 pandas DataFrame
df = pd.DataFrame(sheets[1:], columns=sheets[0])

# 設定 index 為「日期 + 時間」
df["日期時間"] = df["日期"] + " " + df["時間"]
df.set_index("日期時間", inplace=True)

# 顯示前 5 筆
df.head()


,日期,時間,支出,金額
日期時間,,,,
2025-09-11 11:00,2025-09-11,11:00,餐飲,130


In [8]:
import datetime

In [9]:
import datetime
import pytz

# 設定台灣時區
tz = pytz.timezone("Asia/Taipei")

# 支出類別選項
categories = ["餐飲", "交通", "娛樂", "日用品", "其他"]

# 儲存所有支出
records = []
total_amount = 0

while True:
    now = datetime.datetime.now(tz)
    date_str = now.strftime("%Y-%m-%d")
    time_str = now.strftime("%H:%M")  # 24小時制

    # 顯示支出類別選單
    print("\n請選擇支出類別：")
    for i, cat in enumerate(categories, start=1):
        print(f"{i}. {cat}")
    cat_choice = int(input("輸入編號: "))
    category_selected = categories[cat_choice - 1]

    # 輸入金額
    amount = float(input("請輸入金額: "))

    # 存入紀錄
    records.append([date_str, time_str, category_selected, amount])
    total_amount += amount

    print(f"✅ 已記錄：{date_str} {time_str} | {category_selected} | {amount:.2f} 元")

    # 詢問是否繼續
    cont = input("是否繼續輸入支出？(y/n): ")
    if cont.lower() != "y":
        break

# 顯示總結
print("\n📊 支出總結")
for r in records:
    print(f"{r[0]} {r[1]} | {r[2]} | {r[3]:.2f} 元")
print(f"\n💰 總支出：{total_amount:.2f} 元")



請選擇支出類別：
1. 餐飲
2. 交通
3. 娛樂
4. 日用品
5. 其他
輸入編號: 1
請輸入金額: 120
✅ 已記錄：2025-09-17 23:42 | 餐飲 | 120.00 元
是否繼續輸入支出？(y/n): n

📊 支出總結
2025-09-17 23:42 | 餐飲 | 120.00 元

💰 總支出：120.00 元


In [10]:
date_str

'2025-09-17'

In [11]:
time_str

'23:42'

In [12]:
category_selected


'餐飲'

In [13]:
amount

120.0

In [14]:
# 創建一個新的 DataFrame 來代表你要新增的資料
new_data = pd.DataFrame([{
    '日期': date_str,
    '時間': time_str,
    '支出': category_selected,
    '金額': amount
}])

# 使用 concat() 將新資料合併到舊的 df 中
df = pd.concat([df, new_data], ignore_index=True)

In [15]:
df

,日期,時間,支出,金額
0,2025-09-11,11:00,餐飲,130
1,2025-09-17,23:42,餐飲,120.0


In [16]:
# 將 NaN 轉成空字串
df_clean = df.fillna("")

# 將 DataFrame 轉成列表的列表
data_to_write = df_clean.values.tolist()

# 寫入 Google Sheet
worksheet = gsheets.worksheet('工作表1')
worksheet.append_rows(values=data_to_write, value_input_option='USER_ENTERED')
print("資料已成功寫入 Google 工作表！")

資料已成功寫入 Google 工作表！
